# Summary of `xyBnG` results

The summaries are stored in two files. One is ``scenario.par``This is a text file about the scenario parameters. The other is a serialized `DataFrame`, called `summary.ser`. Below are example codes to retrieve the records.

In [ ]:
using DataFrames, Serialization, Statistics, Plots, Dates

In [ ]:
dir = "/mnt/a/store/xybng/cattle"  # modify this to your work directory
pars = readlines("$dir/scenario.par")
start = Dates.unix2datetime(parse(Float64, split(pars[1])[2]))
println("Simulation started: $start")
stop, mature = begin
    if split(pars[end])[1] == "Ended:"
        Dates.unix2datetime(parse(Float64, split(pars[end])[2])), true
    else
        Dates.unix2datetime(mtime("$dir/summary.ser")), false
    end
end
println("Simulation finised: $stop")
println()
if mature
    print(join(pars[2:end-1], '\n'))
else
    print(join(pars[2:end], '\n'))
end

In [ ]:
rst = deserialize("$dir/summary.ser")
describe(rst)

In [ ]:
# What schemes used in the simulatino
schemes = unique(rst.scheme)

In [ ]:
dic = Dict{String, DataFrame}()
for scheme in schemes
    dic[scheme] = select(filter(x -> x.scheme == scheme, rst), Not(:scheme))
end

In [ ]:
# The mean of each generation over repeats
combine(groupby(dic["aaocs"], :grt), Not(:repeat) .=> mean .=> Not(:repeat))
# replace mean to var, you get variances
# similar results can be obtained on other schemes
# plots to be added